In [1]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [2]:
EPOCHS = 1
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_CLASSES = 15

# 1. Loading and Processing Data

In [3]:
# TRAIN_PATH='/kaggle/input/traffic-signs/dataset/train'
# VAL_PATH='/kaggle/input/traffic-signs/dataset/val'
# TEST_PATH='/kaggle/input/traffic-signs/dataset/test'
# CLASS_MAPPING_PATH='/kaggle/input/traffic-signs/dataset/class_mapping.txt'
TRAIN_PATH='../dataset/classification/train'
VAL_PATH='../dataset/classification/val'
TEST_PATH='../dataset/classification/test'
CLASS_MAPPING_PATH='../dataset/classification/class_mapping.txt'

In [4]:
# Hàm đọc class mapping từ file txt
def load_class_mapping(txt_path):
    idx_to_class = {}
    with open(txt_path, 'r') as f:
        for line in f:
            idx, class_name = line.strip().split(':')
            idx_to_class[int(idx)] = class_name.strip()
    return idx_to_class

# Load mapping
idx_to_class = load_class_mapping(CLASS_MAPPING_PATH)
class_to_idx = {v: k for k, v in idx_to_class.items()}  # Đảo ngược để có class_to_idx

In [5]:
class_to_idx

{'speed limit 20': 0,
 'speed limit 30': 1,
 'speed limit 50': 2,
 'speed limit 60': 3,
 'speed limit 70': 4,
 'speed limit 80': 5,
 'no entry for all vehicles': 6,
 'speed limit 100': 7,
 'speed limit 120': 8,
 'no passing': 9,
 'no truck passing': 10,
 'no parking': 11,
 'no horn': 12,
 'no entry in this direction': 13,
 'no cars': 14}

In [6]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load dataset
train_dataset = datasets.ImageFolder(TRAIN_PATH, transform=train_transform)
val_dataset = datasets.ImageFolder(VAL_PATH, transform=val_transform)
test_dataset = datasets.ImageFolder(TEST_PATH, transform=test_transform)

In [7]:
# Ghi đè class names bằng mapping từ file
train_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
train_dataset.class_to_idx = {cls: i for i, cls in enumerate(train_dataset.classes)}

# Ghi đè class names bằng mapping từ file
val_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
val_dataset.class_to_idx = {cls: i for i, cls in enumerate(val_dataset.classes)}

# Ghi đè class names bằng mapping từ file
test_dataset.classes = [idx_to_class[i] for i in sorted(idx_to_class.keys())]
test_dataset.class_to_idx = {cls: i for i, cls in enumerate(test_dataset.classes)}

# Tạo DataLoader (giữ nguyên)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [2]:
print("Train classes:", train_dataset.classes)
print("Class to index mapping:", train_dataset.class_to_idx)

NameError: name 'train_dataset' is not defined

# 2. Model

In [1]:
# load train model pth
model_path='../model/resnet50_model_v01.pth'
def load_model(model_path):
    model = models.resnet50(pretrained=True)
    model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)
    model.load_state_dict(torch.load(model_path))
    return model

model=load_model(model_path)

NameError: name 'models' is not defined

In [36]:
model = models.resnet50(pretrained=True)

# Đóng băng tất cả trừ layer3, layer4 và fc
for name, param in model.named_parameters():
    if "layer3" not in name and "layer4" not in name and "fc" not in name:
        param.requires_grad = False

# Thay đổi lớp FC cuối cho 15 lớp
model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)

D:\chientuhocai\Traffic_Sign\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\chientuhocai\Traffic_Sign\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [37]:
# Chuyển model sang GPU (nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [38]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Lists để lưu metrics
epochs_list = []
train_loss_list = []
val_loss_list = []
accuracy_list = []
precision_list = []
recall_list = []

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    # === Training Phase với Progress Bar ===
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]", leave=False)
    for inputs, labels in train_loop:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        train_loop.set_postfix(loss=loss.item())  # Hiển thị loss hiện tại
    
    # === Validation Phase với Progress Bar ===
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]", leave=False)
    with torch.no_grad():
        for inputs, labels in val_loop:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            val_loop.set_postfix(val_loss=loss.item())  # Hiển thị val_loss hiện tại
    
    # === Tính toán metrics ===
    train_loss /= len(train_dataset)
    val_loss /= len(val_dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    
    # Lưu metrics
    epochs_list.append(epoch + 1)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    
    # In thông tin epoch
    print(f"\nEpoch {epoch+1}/{EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | "
          f"Accuracy: {accuracy:.4f} | "
          f"Precision: {precision:.4f} | "
          f"Recall: {recall:.4f}")

Epoch 1/1 [Train]:  20%|█▉        | 326/1641 [22:23<1:28:13,  4.03s/it, loss=1.34] 

In [ ]:
torch.save(model.state_dict(), "resnet50_model.pth")    

In [ ]:
plt.figure(figsize=(12, 4))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs_list, train_loss_list, label='Train Loss')
plt.plot(epochs_list, val_loss_list, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training & Validation Loss')
plt.legend()

# Plot Metrics
plt.subplot(1, 2, 2)
plt.plot(epochs_list, accuracy_list, label='Accuracy')
plt.plot(epochs_list, precision_list, label='Precision (Macro)')
plt.plot(epochs_list, recall_list, label='Recall (Macro)')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Model Performance Metrics')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
metrics_df = pd.DataFrame({
    'Epoch': epochs_list,
    'Train Loss': train_loss_list,
    'Val Loss': val_loss_list,
    'Accuracy': accuracy_list,
    'Precision': precision_list,
    'Recall': recall_list
})

metrics_df.to_csv('training_metrics.csv', index=False)

In [1]:
# create class mapping txt
class_list=[
    'speed limit 20',
    'speed limit 30',
    'speed limit 50',
    'speed limit 60',
    'speed limit 70',
    'speed limit 80',
    'no entry for all vehicles',
    'speed limit 100',
    'speed limit 120',
    'no passing',
    'no truck passing',
    'no parking',
    'no horn',
    'no entry in this direction',
    'no cars'
]

with open('class_mapping.txt', 'w') as file:
    for i, class_name in enumerate(class_list):
        file.write(f"{i}: {class_name}\n")